# PaddleSeg实战——Matting

![](https://ai-studio-static-online.cdn.bcebos.com/67a920cfa5c541b1bce8dd4bab8d2101f482d216145148f3822fb4feadf3601a)

**Matting**是一种通过计算图像的颜色和透明度从从背景中提取前景对象的技术。 这项技术被广泛用于电影等行业，实现精细地背景替换、图像构图，制造各种惊艳的视觉效果。 

相对于经典的语义分割任务来说，Matting更加注重**抠图效果**是否自然。语义分割主要提供对图像中每个像素点的理解，最终将具有相同语义的像素输出为特定标签，而不注重实际的视觉效果。在很多应用场景中，人们关注的不仅仅是各个语义区域分割结果是否正确，还对观察到的视觉效果是否更加友好提出了更高的要求。新冠疫情期间，学校、企业纷纷使用**线上视频会议**，有时用户需要将自己的真实背景隐藏起来，且希望在发丝处替换背景能更加自然，这便是Matting派上用场的时候了。一些语义分割模型的弊病在于其分割边缘十分粗糙，这在一些对分割精细程度要求较高的场景（如电影的虚拟场景替换）是不能接受的。

Matting可以看作是一个特殊的分割问题，它旨在估计**透明度**，区分`前景`和`背景`，主要目的是拿到前景，是一项真正实用的替换背景技术。分割和Matting的一个重要区别是：分割返回的是像素分类标签，其结果是整型数据；而Matting返回的是属于前景或背景的概率`P`，从而在前景与背景交互区域产生渐变的效果，使得抠图更加自然。Matting分割模型训练完成后，对于原始图像每个位置上的像素，都将生成一个表示其`前景透明度`的值，称为 **Alpha**。 图像中所有 Alpha 值的集合称为 Alpha Matte。最后，利用Alpha将原始图像的对应位置上的像素点做pixel-wise的运算，便可实现精细的背景替换。

本文将带大家完整的跑通Matting全流程。

## 一、前置准备

请按照以下步骤配置相应的环境。

### 1. 下载PaddlePaddle

* 版本要求

  * PaddlePaddle >= 2.0.2

  * Python >= 3.7+


由于图像分割模型计算开销大，推荐在GPU版本的PaddlePaddle下使用PaddleSeg。推荐安装10.0以上的CUDA环境。安装教程请见[PaddlePaddle官网](https://www.paddlepaddle.org.cn/install/quick?docurl=/documentation/docs/zh/install/pip/linux-pip.html)。

注：如果你在AI Studio上运行此项目，不必担心版本问题。

### 2. 下载PaddleSeg
* 如果你的下载进度缓慢，可以考虑先将一份PaddleSeg的副本导入到[gitee](https://gitee.com/)，再从gitee上下载PaddleSeg。
* 如果你在AI Studio上运行此项目，我们建议你在`~/work/`目录下存放文件，以实现持久化操作。

In [1]:
%cd ~/work/

!git clone https://github.com/PaddlePaddle/PaddleSeg.git

/home/aistudio/work
fatal: 目标路径 'PaddleSeg' 已经存在，并且不是一个空目录。


### 3. 安装PaddleSeg

In [2]:
%cd ~/work/PaddleSeg
!pip install -e .
!pip install scikit-image

/home/aistudio/work/PaddleSeg
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining file:///home/aistudio/work/PaddleSeg
  Running setup.py develop for paddleseg
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 13.3MB 6.0MB/s eta 0:00:011
     |████████████████████████████████| 6.2MB 6.0MB/s eta 0:00:01
     |████████████████████████████████| 184kB 3.5MB/s eta 0:00:01


### 4. 准备数据集

我们使用 MODNet 的开源数据集 [PPM-100](https://github.com/ZHKKKe/PPM) 作为本教程的演示数据集。

将数据集组织成如下结构，并将数据集放在数据目录下。

```
PPM-100/
|--train/
|  |--fg/
|  |--alpha/
|
|--val/
|  |--fg/
|  |--alpha
|
|--train.txt
|
|--val.txt
```

可直接下载**整理后的** [PPM-100](https://github.com/ZHKKKe/PPM) 数据进行后续教程。
其中，fg目录下的图象名称需和alpha目录下的名称一一对应。`train.txt`和`val.txt`的内容如下:

```
train/fg/14299313536_ea3e61076c_o.jpg
train/fg/14429083354_23c8fddff5_o.jpg
train/fg/14559969490_d33552a324_o.jpg
...
```

如果完整图象需由前景和背景进行合成的数据集，类似论文 [Deep Image Matting](https://arxiv.org/pdf/1703.03872.pdf) 里使用的数据集Composition-1k，则数据集应整理成如下结构：
```
Composition-1k/
|--bg/
|
|--train/
|  |--fg/
|  |--alpha/
|
|--val/
|  |--fg/
|  |--alpha/
|  |--trimap/ (如果存在)
|
|--train.txt
|
|--val.txt
```

train.txt的内容如下：
```
train/fg/fg1.jpg bg/bg1.jpg
train/fg/fg2.jpg bg/bg2.jpg
train/fg/fg3.jpg bg/bg3.jpg
...
```

val.txt的内容如下, 如果不存在对应的trimap，则第三列可不提供，代码将会自动生成。
```
val/fg/fg1.jpg bg/bg1.jpg val/trimap/trimap1.jpg
val/fg/fg2.jpg bg/bg2.jpg val/trimap/trimap2.jpg
val/fg/fg3.jpg bg/bg3.jpg val/trimap/trimap3.jpg
...
```


In [6]:
# 下载 PPM 数据集。你也可以使用自定义数据集。 notebook输出不能交互，换到终端执行
# %cd ~/work/PaddleSeg/contrib/Matting
# !mkdir data 
%cd data
!wget https://paddleseg.bj.bcebos.com/matting/datasets/PPM-100.zip
# !unzip PPM-100.zip
# %cd ..

/home/aistudio/work/PaddleSeg/contrib/Matting/data
--2022-01-05 20:08:59--  https://paddleseg.bj.bcebos.com/matting/datasets/PPM-100.zip
正在解析主机 paddleseg.bj.bcebos.com (paddleseg.bj.bcebos.com)... 182.61.200.195, 182.61.200.229, 2409:8c04:1001:1002:0:ff:b001:368a
正在连接 paddleseg.bj.bcebos.com (paddleseg.bj.bcebos.com)|182.61.200.195|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 263089716 (251M) [application/octet-stream]
正在保存至: “PPM-100.zip.4”

PPM-100.zip.4       100%[===================>] 250.90M  53.7MB/s    in 5.1s    

2022-01-05 20:09:04 (48.8 MB/s) - 已保存 “PPM-100.zip.4” [263089716/263089716])




## 三、训练
为了加快收敛，下载预训练模型，并在配置文件里传给model下的pretrained。

In [8]:
%cd ~/work/PaddleSeg/contrib/Matting

/home/aistudio/work/PaddleSeg/contrib/Matting


In [9]:
!wget https://paddleseg.bj.bcebos.com/matting/models/modnet-mobilenetv2.pdparams

--2022-01-05 20:13:03--  https://paddleseg.bj.bcebos.com/matting/models/modnet-mobilenetv2.pdparams
正在解析主机 paddleseg.bj.bcebos.com (paddleseg.bj.bcebos.com)... 182.61.200.229, 182.61.200.195, 2409:8c04:1001:1002:0:ff:b001:368a
正在连接 paddleseg.bj.bcebos.com (paddleseg.bj.bcebos.com)|182.61.200.229|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 39162180 (37M) [application/octet-stream]
正在保存至: “modnet-mobilenetv2.pdparams.1”

modnet-mobilenetv2. 100%[===================>]  37.35M  12.1MB/s    in 3.2s    

2022-01-05 20:13:06 (11.7 MB/s) - 已保存 “modnet-mobilenetv2.pdparams.1” [39162180/39162180])



In [15]:
%cd ~/work/PaddleSeg/contrib/Matting
!python train.py \
       --config configs/modnet/modnet_mobilenetv2.yml \
       --iters 50 \
       --batch_size 1 \
       --save_interval 10 \
       --do_eval \
       --log_iters 1 \
       --num_workers 0 \
       --save_dir output

/home/aistudio/work/PaddleSeg/contrib/Matting
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: ResNet18_vd exists already! It is now updated to <function ResNet18_vd at 0x7ff208129b90> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: ResNet50_vd exists already! It is now updated to <function ResNet50_vd at 0x7ff20812f950> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: ResNet101_vd exists already! It is now updated to <function ResNet101_vd at 0x7ff20812fa70> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: MobileNetV2 exists already! It is now updated to <function MobileNetV2 at 0x7ff20812c950> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: HRNet_W18_Small_V1 exists already! It is now updated to <function HRNet_W18_

## 四、评估
`-save_result`开启会保留图片的预测结果，可选择关闭以加快评估速度。

你可以直接下载我们提供的模型进行评估。

更多参数信息请运行如下命令进行查看:

```
python val.py --help
```

In [16]:
!python val.py \
       --config configs/modnet/modnet_mobilenetv2.yml \
       --model_path modnet-mobilenetv2.pdparams \
       --save_dir ./output/results \
       --save_results

/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: ResNet18_vd exists already! It is now updated to <function ResNet18_vd at 0x7f16d5a17b00> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: ResNet50_vd exists already! It is now updated to <function ResNet50_vd at 0x7f16d5a1e8c0> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: ResNet101_vd exists already! It is now updated to <function ResNet101_vd at 0x7f16d5a1e9e0> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: MobileNetV2 exists already! It is now updated to <function MobileNetV2 at 0x7f16d5a198c0> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: HRNet_W18_Small_V1 exists already! It is now updated to <function HRNet_W18_Small_V1 at 0x7f16d5a23440> !!!
  component_na

## 五、预测及可视化结果保存

如模型需要trimap信息，需要通过 `--trimap_path` 传入trimap路径。

你可以直接下载我们提供的模型进行预测。

更多参数信息请运行如下命令进行查看:

```
python predict.py --help
```

In [17]:
!python predict.py \
    --config configs/modnet/modnet_mobilenetv2.yml \
    --model_path modnet-mobilenetv2.pdparams \
    --image_path data/PPM-100/val/fg/ \
    --save_dir ./output/results

/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: ResNet18_vd exists already! It is now updated to <function ResNet18_vd at 0x7f93eb46eb00> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: ResNet50_vd exists already! It is now updated to <function ResNet50_vd at 0x7f93eb4758c0> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: ResNet101_vd exists already! It is now updated to <function ResNet101_vd at 0x7f93eb4759e0> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: MobileNetV2 exists already! It is now updated to <function MobileNetV2 at 0x7f93eb4708c0> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: HRNet_W18_Small_V1 exists already! It is now updated to <function HRNet_W18_Small_V1 at 0x7f93eb47a440> !!!
  component_na

## 六、背景替换

在得到一个性能较好的Matting模型后，就可以用它实现一些有趣的应用了。还记得我们在文章开头提到的人像抠图吗？Matting将帮助你得到更加精细的抠图。

In [18]:
# 执行背景替换

%cd ~/work/PaddleSeg/contrib/Matting/ 

!python bg_replace.py \
    --config configs/modnet/modnet_mobilenetv2.yml \
    --model_path modnet-mobilenetv2.pdparams \
    --image_path data/PPM-100/val/fg/17464489461_0e158881db_o.jpg \
    --save_dir ./output/bg_results \
    # --bg_path data/bg_demo.jpg \

# 可在 ./output/bg_results 路径下查看背景替换效果

/home/aistudio/work/PaddleSeg/contrib/Matting
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: ResNet18_vd exists already! It is now updated to <function ResNet18_vd at 0x7fb8c65a90e0> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: ResNet50_vd exists already! It is now updated to <function ResNet50_vd at 0x7fb8c65aeb90> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: ResNet101_vd exists already! It is now updated to <function ResNet101_vd at 0x7fb8c65aecb0> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: MobileNetV2 exists already! It is now updated to <function MobileNetV2 at 0x7fb8c65a9b90> !!!
  component_name, component))
/home/aistudio/work/PaddleSeg/paddleseg/cvlibs/manager.py:116: UserWarning: HRNet_W18_Small_V1 exists already! It is now updated to <function HRNet_W18_

### 参数说明：
  * config: 模型配置文件存放路径。
  * model_path: 模型参数文件存放路径。
  * image_path: 需要区分前景、背景的原始图片。经处理后，前景（如人像）将保留，背景将被替换。
  * bg_path: 背景图片存放路径。如果不指定，默认为绿色背景。
  * save_dir: 替换背景后的结果存放路径。默认为`./output`。
 

 ## <left> **背景替换效果展示(左侧为原图，右侧为背景替换效果)**
 
 
 <div align="left">
<img src="https://ai-studio-static-online.cdn.bcebos.com/9a0bf85aa4254b82bbdb0d22ad6162f2a0f84e6182394ee98f4c516b3e09b270"  width = "500" />  
</div>
  
<br>
  
 <div align="left">
<img src="https://ai-studio-static-online.cdn.bcebos.com/86836b105d5648dcaa9831423b2a2e8cd1938d46db304741b88f09c888dfd827"  width = "500" />  
</div>

<br>
  
 <div align="left">
<img src="https://ai-studio-static-online.cdn.bcebos.com/fa85fab86c5443ffa9c27c5a39d7f393f0f8fc0c0c67460e80e1e0cec03feb16"  width = "500" />  
</div>